# Generate Start Data

In the absense of having data on hand I will have to generate a bunch of fake data.

In [37]:
import numpy as np
import matplotlib

matplotlib.use('Agg')


import matplotlib.pyplot as plt
import uuid
from control.matlab import *
import os

/usr/lib/python3/dist-packages/matplotlib/__init__.py:1352: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [24]:
# Assume engine is a 1st order transfer function with 1s time constant & unity gain.
# "A horse is a sphere if it makes the equations easier"
sys1 = tf([1],[1,1])

In [25]:
# Generate 5 minutes of data at 0.1s resolution.
dt = 0.1
t0 = 0
tfinal = 300
T = np.arange(t0, tfinal+dt, dt)

In [26]:
def gen_start(T):
    crank_speed = np.random.uniform(200,210)
    idle_speed  = np.random.uniform(595, 605)

    crank_time = np.random.uniform(10,60)
    crank_duration = np.random.uniform(5, 15)
    
    # Simulate a good start.
    U = np.zeros(T.shape)
    U[(T>crank_time)]=crank_speed
    if crank_duration>12:
        U[(T>crank_time+crank_duration)]=0
        start_type = "no"
    else:
        U[(T>crank_time+crank_duration)]=idle_speed
        start_type = "good"
    
    yout, T, xout = lsim(sys1, U=U, T=T)
    
    N = np.random.normal(0, 5, yout.shape)
    yout[U>0]+=N[U>0]
        
    return yout, start_type

In [27]:
# Generate 5 minutes of data at 0.1s resolution.
dt = 0.1
t0 = 0
tfinal = 300
T = np.arange(t0, tfinal+dt, dt)

In [39]:
while True:
    try:
        fig = plt.figure(figsize=(1,1), dpi=32)

        yout, start_type = gen_start(T)
        fig.clf()
        plt.plot(T,yout)
        plt.axis('off')

        base_dir = os.path.join("Images", start_type)
        if not os.path.exists(base_dir):
            os.makedirs(base_dir)

        U = uuid.uuid4()
        img_name = str(U)+".png"
        img_path = os.path.join(base_dir, img_name)
        plt.savefig(img_path)
        plt.close(plt.gcf())
    except KeyboardInterrupt:
        print
    except:
        raise

KeyboardInterrupt: 

In [31]:
fig.clear()